In [1]:
import pandas as pd
import numpy as np
import re
from pymongo import MongoClient
from collections import defaultdict
from nltk.corpus import wordnet as wn
#from transliteration import transliterate_word
import nltk
from nltk import word_tokenize
from nltk.util import ngrams as ngrams_creator
from googletrans import Translator
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 
import pickle
#nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.corpus import sentiwordnet as swn
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ankita/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
translator = Translator(service_urls=['translate.google.co.in'])

In [3]:
'''
Utility function to clean tweet
'''

def clean_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub('https.*$', 'URL', tweet)  # remove URLs
    tweet = re.sub('RT|cc', '', tweet)  # remove RT and cc
    tweet = re.sub('#\S+', '', tweet)  # remove hashtags
    tweet = re.sub('@\S+', '', tweet)  # remove mentions
    tweet = re.sub('[%s]' % re.escape("""!"#$%&'()*+-./:;<=>@[\]^_`{|}~"""), ' ', tweet)  # remove punctuations
    tweet = re.sub('\s+', ' ', tweet) 
    #tweet = word_tokenize(tweet) # remove repeated characters (hellooooo into hello)
    # remove extra whitespace
    return tweet


'''
Utility function to handle commonly used short forms
'''

def handle_short_forms(w):
    if w == 'h':
        return 'hai'
    elif w == 'n':
        return 'na'
    elif w == 'da':
        return 'the'
    elif w == 'wid':
        return 'with'
    elif w == 'pr':
        return 'par'
    elif w == 'mattt':
        return 'mat'
    elif w == 'vo':
        return 'woh'
    elif w == 'ki':
        return 'kee'
    elif w == 'ap':
        return 'aap'
    elif w == 'bs':
        return 'bas'
    elif w == 'goood':
        return 'very good'
    elif w == 'tera':
        return 'teraa'
    elif w == 'cnfsn':
        return 'confusion'
    elif w == 'ka':
        return 'kaa'
    elif w == 'rkhi':
        return 'rakhi'
    elif w == 'thts':
        return 'thats'
    elif w == 'cald':
        return 'called'
    elif w == 'tabhe':
        return 'tabhi'
    elif w == 'pta':
        return 'pata'
    elif w == 'b':
        return 'bhi'
    elif w == 'nai':
        return 'nahi'
    elif w == 'f':
        return 'of'
    elif w == 'd':
        return 'the'
    else:
        return w
    
'''
Translate word . 
'''
def translate(word):
    return translator.translate(word,src='hi' , dest='en').text

'''
Self defined contractions
''' 
def load_dict_contractions():
    
    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "I'd":"I would",
        "I'll":"I will",
        "i'll":"I will",
        "I'm":"I am",
        "I'm'a":"I am about to",
        "I'm'o":"I am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "I've":"I have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "Whatcha":"What are you",
        "whatcha":"What are you",
        "luv":"love",
        "sux":"sucks"
        }

In [ ]:
'''
Utility function to handle commonly mispelled words
'''

def handle_mispelled_forms(w):
    if w == 'koch':
        return 'kuch'
    elif w == 'n':
        return 'na'
    elif w == 'trnt':
        return 'turant'
    elif w == 'kon':
        return 'kaun'
    elif w == 'pr':
        return 'par'
    elif w == 'mattt':
        return 'mat'
    elif w == 'vo':
        return 'woh'
    elif w == 'ki':
        return 'kee'
    elif w == 'ap':
        return 'aap'
    elif w == 'bs':
        return 'bas'
    elif w == 'bkar':
        return 'bekaar'
    elif w == 'teraa':
        return 'tera'
    elif w == 'cnfsn':
        return 'confusion'
    elif w == 'ka':
        return 'kaa'
    elif w == 'rkhi':
        return 'rakhi'
    elif w == 'thts':
        return 'thats'
    elif w == 'cald':
        return 'called'
    elif w == 'tabhe':
        return 'tabhi'
    elif w == 'pta':
        return 'pata'
    elif w == 'b':
        return 'bhi'
    elif w == 'nai' or w== "nhi":
        return 'nahi'
    elif w == 'f':
        return 'of'
    elif w == 'd':
        return 'the'
    else:
        return w

In [4]:
'''
Handling short forms and contractions in the sentences 
'''
long_form_dict = load_dict_contractions()
def expand_sent(sentence):
    final_sent =""
    res = " ".join(long_form_dict.get(ele, ele) for ele in sentence.split()) 
    for word in res.split():
        final_sent += (handle_short_forms(word))+ " "
    return final_sent


In [5]:
expand_sent("are ye kya don't laga rkhi h")

'are ye kya do not laga rakhi hai '

In [6]:
'''
Emoticon processing
'''
emoji_list = pd.read_csv('emoji.csv',sep=",")

def find_emoji(sentence):
    positive = 0
    negative = 0
    neutral  = 0
    sentiment =[]
    for word in sentence.split():
        if not emoji_list[emoji_list['Emoji']==word].empty:
            positive += emoji_list.iloc[emoji_list.index[emoji_list['Emoji'] == word].tolist()[0]]['Positive']
            negative += emoji_list.iloc[emoji_list.index[emoji_list['Emoji'] == word].tolist()[0]]['Negative']
            neutral += emoji_list.iloc[emoji_list.index[emoji_list['Emoji'] == word].tolist()[0]]['Neutral']
    return positive,negative,neutral        

In [7]:
'''
Slang word processing
'''
slang_list = pd.read_csv('Hinglish_Profanity_List.csv',sep=",",header=None)
slang_list.columns=['hindi_word','meaning','rating']

def find_slang(sentence):
    slang_exists = False
    total_rating = 0
    for word in sentence.split():
        if not slang_list[slang_list['hindi_word']==word].empty:
            try:
                total_rating += slang_list.iloc[slang_list.index[slang_list['hindi_word'] == word].tolist()[0]]['rating']
            except:
                total_rating +=0
            slang_exists = True
    try:
        total_rating = int(total_rating)
    except:
        total_rating = 0
    return slang_exists,total_rating

In [8]:
#slang_list.iloc[slang_list.index[slang_list['hindi_word'] == word].tolist()[0]]['rating']

In [9]:

#translate("karna chahiye phir wo post krega")

In [10]:
'''
Prepare dataset for training
Remove hashtags and mentions
'''
train = pd.read_csv("train_14k_split_conll.txt",sep="\t",header= None)
train.columns = ["word", "lang", "sentiment"]
mentions = train.index[(train['word']=='@') | (train['word']=='#')].tolist()
mentions_name = list(map(lambda x : x + 1, mentions)) 
train = train.drop(mentions) 
train = train.drop(mentions_name)

In [14]:
'''
Alignment in proper format
'''
train["sentence"] = (train.word == "meta").cumsum()
trainx = train[train.word != "meta"]
sentiment = train.dropna()
trainx['key'] = (trainx['lang'] != trainx['lang'].shift(1)).astype(int).cumsum() #Check consecutive same lang words

trainx['word'] = trainx.word.astype(str)
word_cluster = pd.DataFrame(trainx.groupby(['key','lang'])['word'].apply(' '.join).reset_index())


In [17]:
#eng = word_cluster["word"].head(50).apply(translate)

In [18]:
'''
Applying translations on chunks of words that are tagged as hindi
Loading this into eng_word_cluster if already done
'''
hindi_word_cluster = word_cluster[(word_cluster["lang"]=="Hin")]

with open("english_cluster.pkl", "rb") as input_file:
    eng_word_cluster = pickle.load(input_file)
#eng_word_cluster = hindi_word_cluster["word"].apply(translate)
eng_word_cluster_reset= pd.DataFrame(eng_word_cluster)
eng_word_cluster_reset = eng_word_cluster_reset.reset_index()
eng_word_cluster_reset.columns = ['key','word']


In [19]:
'''
Joining mixed code and translated version into result
'''
eng_word_cluster_reset['key'] = eng_word_cluster_reset['key']+1
result = pd.merge(word_cluster, eng_word_cluster_reset, on='key',how='left')

#result.word_y.fillna(result.word_x, inplace=True)
#result.loc[result['word_y'].isnull(),'word_y'] = result['word_x']
#result.word_y=result.word_y.fillna(result['word_x'])

In [21]:
'''
Formatting the dataframe , reset indexes for joining properly
'''
result = result.reset_index()
result.word_y.fillna(result.word_x, inplace=True)
#result =result.drop(['level_0','index','word_x'],axis=1)
result =result.drop(['index','word_x'],axis=1)
original = pd.merge(word_cluster,trainx[['sentence','key']],on="key")
converted = pd.merge(result,trainx[['sentence','key']],on="key")
original =original.drop_duplicates()
converted =converted.drop_duplicates()

In [25]:
sentiment = sentiment.reset_index()

In [26]:
'''
Final processed data without extra features
'''
g = converted.groupby("sentence")
h = original.groupby("sentence")
#final =  pd.concat([pd.DataFrame(g).reset_index(), pd.DataFrame(h).reset_index()], axis=1, ignore_index=True)
#g.apply(lambda sdf: " ".join(sdf.word))
#g.apply(lambda sdf: " ".join(sdf[sdf.is_nounverb].word))
df1 = pd.DataFrame({
          "sent": h.apply(lambda sdf: " ".join(sdf.word.astype(str))),
          #"lang": g.apply(lambda sdf: " ".join(sdf.lang.astype(str))),
          
  })
df2 = pd.DataFrame({
          "sent": g.apply(lambda sdf: " ".join(sdf.word_y.astype(str))),
          #"lang": g.apply(lambda sdf: " ".join(sdf.lang.astype(str))),
          
  })
df1 = df1.reset_index()
df2 = df2.reset_index()
final = pd.merge(df1,df2,on="sentence", validate="one_to_one")

In [39]:
final

,sentence,sent_x,sent_y
0,1,nen á vist bolest vztek smutek zmatek osam ě lost beznad ě j a nakonec jen klid Asi takhle vypad á m ů j life ...,nen á vist bolest vztek smutek zmatek osam ě lost beznad ě J a nakonec jen klid asi takhle vypad á M ů j life ...
1,2,Haan yaar neha 😔😔 kab karega woh post 😭 Usne na sach mein photoshoot karna chahiye phir woh post karega … https // tco / 5RSlSbZNtt television media congress,Yes friend 😔😔 When will he post 😭 He didn't really photoshoot Then he will post … https // tco / 5RSlSbZNtt television media congress
2,3,5RSlSbZNtt television media congress ke liye nhi h . Ye toh aapko pata chal hi gya hoga . Achha hoga ki Congress ke … https // t . co / HmH8M7PTaK All India,5RSlSbZNtt television media congress Not for . You must have known this . Good that congress ke … https // t . CO / HmH8M7PTaK All India
3,4,HmH8M7PTaK All India me nrc lagu kare w Kashmir se dhara 370ko khatam kare ham Indian ko apse yahi umid hai,HmH8M7PTaK All India me Apply nurse w From Kashmir dhara 370ko Finish we Indian hope this from you
4,5,_ who Pagal hai kya ? They aren ’ t real issues Mandir is important Hindu khatre mei jo hai !,_ who Is crazy ? They aren ’ t real issues Mandir is important Hindu Who is in danger !
...,...,...,...
6132,6133,N9MjA65ZnK Saale Gaandu khud porn dikhta hai aur dusre ko blame karta h 😂,N9MjA65ZnK Saale Lady herself porn Looks at the other blame Does 😂
6133,6134,so i couldn't get no one to cover me for my birthday / but im going to open that day so i mean i guess it's okkk,s / o i couldn't get no one to cover me for my birthday / but im going to open that day so i mean i guess it's Jock
6134,6135,RT Aap logo ki baat nahi kar raha Najim bhai Media walo ki kar raha hu jo bina janche parkhe one sided news … __,"RT Aap Nazim Bhai is not talking about the people of the media, who is testing without checking one sided news … __"
6135,6136,… __ Jay Jay Shree Ram ram rajye aaya he danavo rakshasho chudelno or surpankha ke gharoke sa … https // t . co / mDn0ARJnmM,"… __ Jay Jai Shri Ram, the kingdom of Rama has come with the house of demons, demons and Churnelo … https // t . co / mDn0ARJnmM"


In [46]:
'''
Final data with sentiment
'''
df_merged=pd.concat([final,sentiment.sentiment],axis=1,ignore_index=True)


In [47]:
'''
Rename columns
'''
df_merged = df_merged.rename(columns={0: "id", 1: "sentence_mixed", 2:"sentence_eng",3:"sentiment"})
df_merged= df_merged.drop(columns="id")
df_merged

In [49]:
'''
Expand sentences , replace words 
'''
df_merged['sentence_mixed'] = df_merged['sentence_mixed'].apply(expand_sent)
df_merged['sentence_eng'] = df_merged['sentence_eng'].apply(expand_sent)

In [50]:
df_merged

,sentence_mixed,sentence_eng,sentiment
0,nen á vist bolest vztek smutek zmatek osam ě lost beznad ě j a nakonec jen klid Asi takhle vypad á m ů j life ...,nen á vist bolest vztek smutek zmatek osam ě lost beznad ě J a nakonec jen klid asi takhle vypad á M ů j life ...,neutral
1,Haan yaar neha 😔😔 kab karega woh post 😭 Usne na sach mein photoshoot karna chahiye phir woh post karega … https // tco / 5RSlSbZNtt television media congress,Yes friend 😔😔 When will he post 😭 He did not really photoshoot Then he will post … https // tco / 5RSlSbZNtt television media congress,neutral
2,5RSlSbZNtt television media congress ke liye nhi hai . Ye toh aapko pata chal hi gya hoga . Achha hoga kee Congress ke … https // t . co / HmH8M7PTaK All India,5RSlSbZNtt television media congress Not for . You must have known this . Good that congress ke … https // t . CO / HmH8M7PTaK All India,negative
3,HmH8M7PTaK All India me nrc lagu kare w Kashmir se dhara 370ko khatam kare ham Indian ko apse yahi umid hai,HmH8M7PTaK All India me Apply nurse w From Kashmir dhara 370ko Finish we Indian hope this from you,positive
4,_ who Pagal hai kya ? They aren ’ t real issues Mandir is important Hindu khatre mei jo hai !,_ who Is crazy ? They aren ’ t real issues Mandir is important Hindu Who is in danger !,neutral
...,...,...,...
6132,N9MjA65ZnK Saale Gaandu khud porn dikhta hai aur dusre ko blame karta hai 😂,N9MjA65ZnK Saale Lady herself porn Looks at the other blame Does 😂,negative
6133,so i could not get no one to cover me for my birthday / but im going to open that day so i mean i guess it is okkk,s / o i could not get no one to cover me for my birthday / but im going to open that day so i mean i guess it is Jock,positive
6134,RT Aap logo kee baat nahi kar raha Najim bhai Media walo kee kar raha hu jo bina janche parkhe one sided news … __,"RT Aap Nazim Bhai is not talking about the people of the media, who is testing without checking one sided news … __",positive
6135,… __ Jay Jay Shree Ram ram rajye aaya he danavo rakshasho chudelno or surpankha ke gharoke sa … https // t . co / mDn0ARJnmM,"… __ Jay Jai Shri Ram, the kingdom of Rama has come with the house of demons, demons and Churnelo … https // t . co / mDn0ARJnmM",positive


In [51]:
'''
Clean sentences
'''
df_merged['sentence_mixed'] = df_merged['sentence_mixed'].apply(clean_tweet)
df_merged['sentence_eng'] = df_merged['sentence_eng'].apply(clean_tweet)

In [52]:
pd.options.display.max_colwidth = 400
with open(r"processed_english.pkl", "wb") as output_file:
    pickle.dump(df_merged, output_file)

In [54]:
df_merged.tail(50)

,sentence_mixed,sentence_eng,sentiment
6087,gaandu modi ji ke jeet ke liye sub ko bahut bahut subhkamnaye india win with narendra damodar das modi ready to develo … URL,good luck to subad for the victory of gandu modi ji india win with narendra damodar das modi ready to develo … URL,positive
6088,rt dada saheb phalke international film festival award 2019 yeh award har middle class family ke us talent ko samarpit jo s …,rat dada saheb phalke international film festival award 2019 yeh award har middle class family of that talent ko samarpit jo s …,positive
6089,love jihad dharmantaran to khule aam ho raha hai fir kyu hinduo ke har tyohar par aap … URL,love jihad conversion is becoming more common then why on every festival of hindus aap … URL,negative
6090,ijtnoigpt3 ye kya bakwas hai kaam dhanda or nahi hai kya practice kro yarrrr there is no space science,ijtonoigpt 3 what the hell is that no more business practice do it yarrrr there is no space science,neutral
6091,yarrrr there is no space science sir for finding dowry case is true na false plz save young india jai jawan kar har maidaaan fateh bandeya,yarrrr there is no space science sir for finding dowry case is true na false plz save young india jai jawan kar har maidan fateh bandeya,neutral
6092,jai jawan kar har maidaaan fateh bandeya kar har maidaan fateh URL,jai jawan kar har maidan fateh bandeya do every field fateh URL,positive
6093,yogendrayadav itne saal congress ko chunte rahe to hum log samajhdar nahi hue bjp ko vote dete he bew … URL,yogendraadava so many years congress we did not get smarter if we chose bjp let s vote bew … URL,positive
6094,sir assam rifles best force phir ek bar kashmir me bhejna chahiye inke totte udd jay … URL,sir assam rifles best force should be sent to kashmir once again his tattoos … URL,negative
6095,oqfaqanazm rt dhan dhan satguru tera hi aasra 🙏 bahut aha likha hai sister ji papa saint ram rahim ji bless you with fam …,blasphemous dhan dhan satguru tera hi aasara 🙏 very well written sister ji saint ram mr rahim bless you with fam …,positive
6096,rt ¡ no temas ¡ persevera “ el se ñ or tu dios est á en medio de ti valiente y salvador se alegra y goza contigo te renueva …,rt ¡ no temas ¡ persevera “ el se ñ or tu dios est á en medio de ti valiente y salvador se alegra y goza contigo te renueva …,neutral


In [34]:
'''
Add slang features
'''
df_merged['slang_exists']= df_merged['sentence_mixed'].apply(find_slang)
df_merged[['slang_existance', 'slang_rating']] = pd.DataFrame(df_merged['slang_exists'].tolist(), index=df_merged.index) 
df_merged =df_merged.drop(columns=['slang_exists'])

In [36]:
'''
Add emoji features
'''
df_merged['emoji_sentiment'] = df_merged['sentence_mixed'].apply(find_emoji)
df_merged[['positive_emoji', 'negative_emoji','neutral_emoji']] = pd.DataFrame(df_merged['emoji_sentiment'].tolist(), index=df_merged.index) 
df_merged =df_merged.drop(columns=['emoji_sentiment'])

In [37]:
'''
Dump for further analysis
'''
df_merged.tail(50)
with open(r"processedtillnow.pkl", "wb") as output_file:
    pickle.dump(df_merged, output_file)

In [38]:
'''
Add sentiment intensity analyzer to required models
'''
sid = SentimentIntensityAnalyzer()

ss = sid.polarity_scores(" i am awful 🙂 maybe now i don't even want to understand right or wrong me some ")
for k in sorted(ss):
    print('{0}: {1}, '.format(k, ss[k]), end='')

compound: -0.7447, neg: 0.4, neu: 0.6, pos: 0.0, 

**TODO**

- Smiley check (should be a very main focus as data-set is filled with them)- DONE
- Hindi words transliterated,english meanings derived maybe -DONE
- Clean urls -DONE (REPLACED WITH TERM 'URL')
- Try with translation via google, results were good as each sentence has a mini sequence of hindi words usually followed with english -DONE
- Use the slang dataset maybe as a feature (for negative) -DONE
- https://gist.github.com/mkulakowski2/4289437
- Hindi sentiment analysis papers - very useful
- Sentiword net hindi by amitava das https://amitavadas.com/sentiwordnet.php

**REFERENCES**

- Construction and analysis of Emoji Sentiment Ranking is described in the following paper:
  P. Kralj Novak, J. Smailovic, B. Sluban, I. Mozetic,
  Sentiment of Emojis, PLoS ONE 10(12): e0144296, doi:10.1371/journal.pone.0144296, 2015.
 
- Conference Proceedings Did you offend me? Classification of Offensive Tweets in Hinglish Language
  Puneet Mathur, Ramit Sawhney, Meghna Ayyar, Rajiv Shah 
  
-https://www.dcs.bbk.ac.uk/~ROGER/wikipedia.dat

